1.Check match-level reconciliation
Join bronze.innings to bronze.matches and ensure winner logic is consistent:

If matches.result = 'runs', then inning1_runs_total != inning2_runs_total and winner aligns with higher total (consider toss decision for which team batted first).

If matches.result = 'wickets', the chasing side won; confirm it’s the side batting second (you can infer batting order by inning_no).

In [5]:
## one time setup cell 
from logging import root
from pathlib import Path
from pyspark.sql import functions as F
import sys
import os

current_dir = os.path.abspath('')
# Add parent directory to Python path to allow module imports
sys.path.append(os.path.abspath(os.path.join(current_dir,'..','..')))

from etl.common import get_spark, load_yml, project_root

root = project_root()
cfg = load_yml(str(root / "configs" / "bronze_config.yml"))
fmt = cfg["storage"]["format"]

spark = get_spark("bronze-d-mini-challenges")
## read all the processed data stored in the parquet files
matches = spark.read.format(fmt).load(str(root / cfg["tables"]["matches"]["target_path"]))
deliveries = spark.read.format(fmt).load(str(root / cfg["tables"]["deliveries"]["target_path"]))
innings = spark.read.format(fmt).load(str(root / cfg["tables"]["innings"]["target_path"]))

Challenge 1 — Reconcile winner logic using innings totals

Goal: Check that the recorded winner in matches agrees with runs totals from innings.

Approach:

Build a (match_id → first_batting_team, second_batting_team) map from deliveries.

Pivot innings totals to compare inning1 vs inning2 totals.

Verify:

If result == 'runs': winner is the team with higher total (first or second batting team).

If result == 'wickets': winner is second_batting_team (chasing side).

In [8]:
from pandas import pivot

# 1) Who batted first/second, inferred from deliveries
bat_order = (deliveries
             .groupBy("match_id","inning_no","batting_team")
             .count()
             .groupBy("match_id")
             .pivot("inning_no")
             .agg(F.first("batting_team"))
             .withColumnRenamed("1","first_batting_team")
             .withColumnRenamed("2","second_batting_team"))
# 2) Totals per inning
inng_pivot =(innings
             .groupBy("match_id")
             .pivot("inning_no")
             .agg(F.first("runs_total").alias("runs"))
             .withColumnRenamed("1","innings1_runs")
             .withColumnRenamed("2","innings2_runs"))

# 3) Join with matches
m = (matches
     .select("match_id","winner","result","season","team1","team2")
     .join(bat_order, "match_id","left")
     .join(inng_pivot,"match_id","left"))

# 4) Compute “winner by totals” for runs case
m_chk = (m
    .withColumn("winner_by_runs_totals",
        F.when(F.col("innings1_runs") > F.col("innings2_runs"), F.col("first_batting_team"))
         .when(F.col("innings2_runs") > F.col("innings1_runs"), F.col("second_batting_team"))
         .otherwise(F.lit(None)))
    .withColumn("winner_expected_if_wickets", F.col("second_batting_team")))

#Violations
viol_runs = (m_chk
             .filter(F.col("result") == "runs")
             .filter(F.col("winner") != F.col("winner_by_runs_totals")))

viol_wkts = (m_chk
             .filter(F.col("result") == "wickets")
             .filter(F.col("winner") != F.col("winner_expected_if_wickets")))

print("Runs-result mismatches:", viol_runs.count())
print("Wickets-result mismatches:", viol_wkts.count())

# Peek a few if any
if viol_runs.count() > 0:
    viol_runs.select("match_id","season","winner","winner_by_runs_totals","innings1_runs","innings2_runs").show(5, False)
if viol_wkts.count() > 0:
    viol_wkts.select("match_id","season","winner","second_batting_team").show(5, False)


Runs-result mismatches: 4
Wickets-result mismatches: 0
+----------------+------+-----------------------+---------------------+-------------+-------------+
|match_id        |season|winner                 |winner_by_runs_totals|innings1_runs|innings2_runs|
+----------------+------+-----------------------+---------------------+-------------+-------------+
|ef4334799a0f7dc3|2011  |Kolkata Knight Riders  |Chennai Super Kings  |114          |61           |
|002e76ed7ecda0fe|2016  |Rising Pune Supergiants|Sunrisers Hyderabad  |118          |94           |
|aa68c59a08b2befa|2016  |Rising Pune Supergiants|Delhi Daredevils     |121          |76           |
|f8a03c0a73c274b2|2009  |Kolkata Knight Riders  |Kings XI Punjab      |158          |79           |
+----------------+------+-----------------------+---------------------+-------------+-------------+



Challenge 2 — Find shortened innings (balls_legal < 120)

Goal: Spot rain-shortened/all-out innings.

Approach: balls_legal already computed in Bronze innings; 20 overs × 6 = 120 legal balls.

In [9]:
short = innings.filter(F.col("balls_legal") < 120)

print("Shortened innings count:", short.count())
print("Shortened innings by season:")
(short.groupBy("season")
      .count()
      .orderBy("season")
      .show(50, truncate=False))

# Optional: top 10 most shortened innings
(short
 .select("match_id","season","inning_no","batting_team","balls_legal","runs_total","wickets")
 .orderBy("balls_legal")
 .show(10, truncate=False))

Shortened innings count: 835
Shortened innings by season:
+------+-----+
|season|count|
+------+-----+
|NULL  |136  |
|2009  |42   |
|2011  |53   |
|2012  |43   |
|2013  |57   |
|2014  |49   |
|2015  |36   |
|2016  |46   |
|2017  |45   |
|2018  |46   |
|2019  |47   |
|2021  |41   |
|2022  |47   |
|2023  |39   |
|2024  |51   |
|2025  |57   |
+------+-----+

+----------------+------+---------+---------------------+-----------+----------+-------+
|match_id        |season|inning_no|batting_team         |balls_legal|runs_total|wickets|
+----------------+------+---------+---------------------+-----------+----------+-------+
|fa7b050f710c57d6|NULL  |4        |Delhi Capitals       |2          |3         |0      |
|03aeb71cad4091e0|2019  |4        |Mumbai Indians       |3          |9         |0      |
|f68fbe5d1eee9c7f|2015  |4        |Rajasthan Royals     |3          |6         |2      |
|0f9012e292c1bc2d|NULL  |3        |Sunrisers Hyderabad  |3          |2         |2      |
|fa7b050f710c57d6|

Challenge 3 — (Optional) Derive 1st/2nd batting teams and preview

Goal: Derive and view batting order (useful for later Silver enrichment).

Approach: Same bat_order as in Challenge 1; just preview cleanly.

In [ ]:
print("Batting order sample:")
(bat_order
 .select("match_id","first_batting_team","second_batting_team")
 .orderBy("match_id")
 .show(10, truncate=False))
#If you want these columns in a table, 
#you’ll add them later in Silver as a normalized enrichment.

Batting order sample:
+----------------+---------------------------+-----------------------+
|match_id        |first_batting_team         |second_batting_team    |
+----------------+---------------------------+-----------------------+
|000d25bcc589a749|Royal Challengers Bangalore|Mumbai Indians         |
|002e76ed7ecda0fe|Sunrisers Hyderabad        |Rising Pune Supergiants|
|00649ef73b4016da|Delhi Daredevils           |Kolkata Knight Riders  |
|00727a2982bdca97|Rajasthan Royals           |Mumbai Indians         |
|0082541530f1e37f|Chennai Super Kings        |Kolkata Knight Riders  |
|0093f24d081b16f1|Kings XI Punjab            |Chennai Super Kings    |
|00f5af9ac185847d|Kolkata Knight Riders      |Kings XI Punjab        |
|017c30ad4c4c432d|Royal Challengers Bangalore|Kolkata Knight Riders  |
|019d0f408f294b93|Mumbai Indians             |Gujarat Titans         |
|01a32c4921804c4d|Delhi Capitals             |Lucknow Super Giants   |
+----------------+---------------------------+---------

Tiny DQ script — Reconcile innings vs deliveries (facts = rollups)

Goal: For each (match_id, inning_no), confirm that:

sum(deliveries.runs_total) equals innings.runs_total

sum(deliveries.wicket_fell) equals innings.wickets

Approach: Aggregate deliveries at (match, inning) and join with innings. Fail if any mismatch.

In [11]:
# Aggregate facts from deliveries
facts = (deliveries
    .groupBy("match_id", "inning_no")
    .agg(
        F.sum("runs_total").cast("int").alias("runs_from_deliveries"),
        F.sum(F.col("wicket_fell").cast("int")).cast("int").alias("wickets_from_deliveries")
    ))

# Join with innings rollups
recon = (facts
    .join(innings.select("match_id","inning_no","runs_total","wickets"), ["match_id","inning_no"], "inner")
    .withColumn("runs_match", F.col("runs_from_deliveries") == F.col("runs_total"))
    .withColumn("wkts_match", F.col("wickets_from_deliveries") == F.col("wickets")))

# Count mismatches
bad_runs = recon.filter(~F.col("runs_match")).count()
bad_wkts = recon.filter(~F.col("wkts_match")).count()

print(f"Runs mismatches: {bad_runs}, Wickets mismatches: {bad_wkts}")

# Peek a few problematic rows, if any
if bad_runs or bad_wkts:
    (recon
     .filter((~F.col("runs_match")) | (~F.col("wkts_match")))
     .select("match_id","inning_no","runs_from_deliveries","runs_total","wickets_from_deliveries","wickets")
     .orderBy("match_id","inning_no")
     .show(10, truncate=False))

# Hard assertions (turn this into a test if you like)
assert bad_runs == 0, "Innings runs do not match sum of deliveries for some innings"
assert bad_wkts == 0, "Innings wickets do not match sum of deliveries for some innings"


Runs mismatches: 0, Wickets mismatches: 0
